In [2]:
from gensim.parsing.preprocessing import stem_text
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim import corpora, models 
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
sys.path.append('../')
from utils import word_frequency_per_player, remove_similar_rows_per_player
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import pyLDAvis
from pyLDAvis.gensim_models import prepare
import re




/opt/homebrew/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Load data and convert to stemmed and BERT data 

In [3]:
# load data and drop NA's
df = pd.read_csv('https://github.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/raw/main/Preprocessing/data_clean/de_clean_2.csv')
df = df.dropna(subset=['data'])
df = remove_similar_rows_per_player(df, df['player'])

## Because the playernames took a huge influence on the clustering they will be removed for each player

In [ ]:
df_stem['data'].iloc[1]

'sechser droht viereinhalb monaten paus bitter abschi leverkusen dafur erscheint verletzungspaus lang zudem konkurrenzsitu doppelsech seit woch voll saft stehenden palacio amiri demirbai andrich voraussetzungen extrem schwierig fur aranguiz zuletzt aufgrund personellen engpass innenverteidigung zentral abwehrkett eingesetzt andrich kunftig mittelfeld auflaufen durft alonso vorrucken jahrigen sech sonntag hertha bsc nicht sicher einstufen mocht'

In [ ]:
# for every player remove their names from the texts 
for player in df_stem['player'].unique():
    f_l_name = player.split()

    # Extracting the first name
    first_name = str(f_l_name[0])

    # Extracting the last name
    last_name = str(f_l_name[1])

    updated_pattern = r"\b(" + first_name.lower() + r"|" + last_name.lower() + r")\b|"


    # Apply the function to the data column
    df_stem['data'] = df_stem['data'].apply(lambda x: remove_words(str(x)))
    df_stem.loc[df_stem['player'] == player, 'data'] = df_stem.loc[df_stem['player'] == player, 'data'].apply(lambda x: re.sub(updated_pattern, "", str(x)))




In [ ]:
df_stem['data'].iloc[1]

'sechser droht viereinhalb monaten paus bitter abschi leverkusen dafur erscheint verletzungspaus lang zudem konkurrenzsitu doppelsech seit woch voll saft stehenden palacio amiri demirbai andrich voraussetzungen extrem schwierig fur aranguiz zuletzt aufgrund personellen engpass innenverteidigung zentral abwehrkett eingesetzt andrich kunftig mittelfeld auflaufen durft alonso vorrucken jahrigen sech sonntag hertha bsc nicht sicher einstufen mocht'

In [ ]:
'''
# Function to remove specific words from the string
def remove_words(text):
    pattern = r"\b(mitchel|bakker|exequiel|palacios|piero|hincapie|jeremie|frimpong|jonathan|tah|moussa|diaby|mykhaylo|mudryk)\b"
    return re.sub(pattern, "", text)

# Apply the function to the data column
df_stem['data'] = df_stem['data'].apply(lambda x: remove_words(str(x)))

df_stem
'''


'\n# Function to remove specific words from the string\ndef remove_words(text):\n    pattern = r"\x08(mitchel|bakker|exequiel|palacios|piero|hincapie|jeremie|frimpong|jonathan|tah|moussa|diaby|mykhaylo|mudryk)\x08"\n    return re.sub(pattern, "", text)\n\n# Apply the function to the data column\ndf_stem[\'data\'] = df_stem[\'data\'].apply(lambda x: remove_words(str(x)))\n\ndf_stem\n'

# Stemming and Vectorization

In [ ]:
# create two copies 
df_stem = df.copy()

# create stemmed data
df_stem['data'] = df_stem['data'].apply(stem_text)

df_stem

,data,player,language,publishedAt
0,trainer alonso mitgereisten fan monaco glucksm...,exequiel palacios,de,2023-02-24T09:33:31Z
1,sechser droht viereinhalb monaten paus bitter ...,exequiel palacios,de,2023-03-03T21:35:13Z
2,leverkusen sechser erklart budapest spiel jung...,exequiel palacios,de,2023-03-07T11:34:39Z
3,bakker mittwoch abschluss train leverkusen nic...,exequiel palacios,de,2023-03-08T14:25:18Z
4,warum sehe faz net nicht allerd andrich argent...,exequiel palacios,de,2023-03-09T19:53:46Z
...,...,...,...,...
80,bayer leverkusen wehrt europa leagu lang gut s...,piero hincapie,de,2023-05-12T10:02:12Z
81,alonso taktik duell alten lehrmeist jose mouri...,piero hincapie,de,2023-05-11T21:17:00Z
82,trotz pleit halbfin hinspiel rom gibt leverkus...,piero hincapie,de,2023-05-11T21:09:16Z
83,hitzigen aufeinandertreffen stadio olimpico na...,piero hincapie,de,2023-05-11T20:58:58Z


In [ ]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=0.02)
df_tfidf = tfidf.fit_transform(df_stem.data).toarray()

In [ ]:
df_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
#Store the frequency matrix in data_tfidf. 
data_tfidf = pd.DataFrame(df_tfidf, columns=tfidf.vocabulary_.keys())

#data_tfidf['target_names']=sorted_df_ng.target_names.values #we can't use that because otherwise knn wouldn't work
data_tfidf.head()

,trainer,alonso,fan,monaco,palacio,tor,euro,geht,bundesliga,sonntag,...,freistehend,anzukurbeln,tschechen,anspruchsvol,gehalten,vergaben,lienhart,kleinen,winter,lautstark
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.195206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Store the feature names in a words list.
words = data_tfidf.columns.tolist()[:-2] 
words

['trainer',
 'alonso',
 'fan',
 'monaco',
 'palacio',
 'tor',
 'euro',
 'geht',
 'bundesliga',
 'sonntag',
 'uhr',
 'freiburg',
 'sechser',
 'droht',
 'monaten',
 'paus',
 'leverkusen',
 'dafur',
 'lang',
 'zudem',
 'seit',
 'woch',
 'voll',
 'amiri',
 'demirbai',
 'andrich',
 'extrem',
 'fur',
 'zuletzt',
 'zentral',
 'mittelfeld',
 'durft',
 'jahrigen',
 'sech',
 'hertha',
 'bsc',
 'nicht',
 'sicher',
 'mocht',
 'erklart',
 'budapest',
 'spiel',
 'bayer',
 'donnerstag',
 'hinspiel',
 'europa',
 'leagu',
 'sah',
 'defens',
 'mittelfeldspiel',
 'erfolg',
 'dritt',
 'gelb',
 'kart',
 'gesperrt',
 'werkself',
 'phase',
 'saison',
 'uberzeugt',
 'fussbal',
 'spielt',
 'ware',
 'wahrscheinlich',
 'daher',
 'halbfinal',
 'sorgen',
 'jahrig',
 'bakker',
 'abschluss',
 'train',
 'jedoch',
 'startelf',
 'steht',
 'stehen',
 'bereit',
 'frimpong',
 'wegen',
 'problem',
 'minuten',
 'platz',
 'mussen',
 'gut',
 'trainiert',
 'fuhlt',
 'besser',
 'dabei',
 'grossen',
 'da',
 'gegner',
 'sehe',
 '

In [ ]:
kmeans = KMeans(n_clusters = 4, max_iter=1000, random_state=42)
kmeans.fit(data_tfidf)

/opt/homebrew/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(max_iter=1000, n_clusters=4, random_state=42)

In [ ]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]

In [ ]:
for num, centroid in enumerate(common_words):
    words_in_centroid = [words[word] for word in centroid if word < len(words)]
    print(f"{num}: {', '.join(words_in_centroid)}")
# By adding the if word < len(words) condition, we ensure that only valid indices are used to access the words list or dictionary.

0: sechser, sogar, besser, millionen, auf, halben, gelb, england, spielen, leagu
1: nachspielzeit, gelb, schwaben, gute, tore, bakker, sah, entscheidend, meist, liga
2: sogar, auf, gelb, halben, lang, sieg, abstaub, gute, nehmen, wurden
3: fast, male, durchau, sekund, siegesseri, halben, bereit, strafstoss, deutschen, allerd


In [ ]:
df_stem['cluster'] = kmeans.labels_
df_stem

,data,player,language,publishedAt,cluster
0,trainer alonso mitgereisten fan monaco glucksm...,exequiel palacios,de,2023-02-24T09:33:31Z,2
1,sechser droht viereinhalb monaten paus bitter ...,exequiel palacios,de,2023-03-03T21:35:13Z,2
2,leverkusen sechser erklart budapest spiel jung...,exequiel palacios,de,2023-03-07T11:34:39Z,2
3,bakker mittwoch abschluss train leverkusen nic...,exequiel palacios,de,2023-03-08T14:25:18Z,2
4,warum sehe faz net nicht allerd andrich argent...,exequiel palacios,de,2023-03-09T19:53:46Z,3
...,...,...,...,...,...
80,bayer leverkusen wehrt europa leagu lang gut s...,piero hincapie,de,2023-05-12T10:02:12Z,2
81,alonso taktik duell alten lehrmeist jose mouri...,piero hincapie,de,2023-05-11T21:17:00Z,2
82,trotz pleit halbfin hinspiel rom gibt leverkus...,piero hincapie,de,2023-05-11T21:09:16Z,2
83,hitzigen aufeinandertreffen stadio olimpico na...,piero hincapie,de,2023-05-11T20:58:58Z,2


In [ ]:
clusters = df_stem.groupby(['cluster']).size()
clusters

cluster
0     9987
1     3114
2    42915
3     4609
dtype: int64

# LDA Model

In [ ]:
# Split the texts into a list of words
texts = [text.split() for text in df_stem['data']]

# Create the Gensim dictionary
dictionary = corpora.Dictionary(texts)

# Remove rare and common words from the dictionary
dictionary.filter_extremes(no_below=118, no_above=0.95)

# Derive the absolute frequency matrix using doc2bow
corpus = [dictionary.doc2bow(text) for text in texts]


In [ ]:
lda = models.LdaModel(corpus, num_topics=4, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)


In [ ]:
lda.show_topics()

[(0,
  '0.027*"bayer" + 0.022*"leagu" + 0.022*"leverkusen" + 0.022*"wirtz" + 0.020*"europa" + 0.019*"union" + 0.018*"seit" + 0.015*"tah" + 0.014*"hinspiel" + 0.014*"nicht"'),
 (1,
  '0.022*"leverkusen" + 0.017*"fur" + 0.017*"uber" + 0.016*"bayer" + 0.014*"nicht" + 0.012*"freiburg" + 0.012*"demirbai" + 0.011*"hincapi" + 0.011*"traf" + 0.011*"frimpong"'),
 (2,
  '0.019*"leverkusen" + 0.017*"hincapi" + 0.014*"bayer" + 0.013*"fur" + 0.012*"alonso" + 0.011*"spiel" + 0.009*"adli" + 0.009*"mourinho" + 0.009*"minut" + 0.007*"weigl"'),
 (3,
  '0.035*"fur" + 0.029*"hincapi" + 0.023*"leverkusen" + 0.023*"nicht" + 0.018*"alonso" + 0.015*"bayer" + 0.011*"trainer" + 0.009*"bochum" + 0.008*"rolf" + 0.008*"innenverteidig"')]

In [ ]:
topics=lda[corpus]
df_stem['lda_predicted_clusters']= [max(topics[i],key=lambda item:item[1])[0] for i in range(len(topics))]

In [ ]:
df_stem.head()

,data,player,language,publishedAt,cluster,lda_predicted_clusters
0,trainer alonso mitgereisten fan monaco glucksm...,exequiel palacios,de,2023-02-24T09:33:31Z,2,3
1,sechser droht viereinhalb monaten paus bitter ...,exequiel palacios,de,2023-03-03T21:35:13Z,2,3
2,leverkusen sechser erklart budapest spiel jung...,exequiel palacios,de,2023-03-07T11:34:39Z,2,3
3,bakker mittwoch abschluss train leverkusen nic...,exequiel palacios,de,2023-03-08T14:25:18Z,2,3
4,warum sehe faz net nicht allerd andrich argent...,exequiel palacios,de,2023-03-09T19:53:46Z,3,3


In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(lda, corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.265657  0.155978       1        1  28.924360
2     -0.040156 -0.154495       2        1  25.361600
1      0.065382 -0.203309       3        1  22.980326
0      0.240431  0.201827       4        1  22.733715, topic_info=         Term          Freq         Total Category  logprob  loglift
193     union  21858.000000  21858.000000  Default  30.0000  30.0000
73     europa  30888.000000  30888.000000  Default  29.0000  29.0000
48       seit  23769.000000  23769.000000  Default  28.0000  28.0000
83   hinspiel  16395.000000  16395.000000  Default  27.0000  27.0000
89      leagu  35865.000000  35865.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
477      uber   7390.726789  36870.682534   Topic4  -5.0191  -0.1259
159    berlin   6147.554682  10885.622845   Topic4  -5.2033   0.9099
115  frimpong   6464.874590  27091.683415   Topic4  -5.1529   0.0485
313     diabi   6404.025578  26501.897581   Topic4  -5.1624   0.0610
11        tor   6137.343244  22948.832050   Topic4  -5.2049   0.1624

[298 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1933      1  0.999607    ,,e
1933      4  0.000369    ,,e
535       1  0.999786  ,,wir
535       4  0.000222  ,,wir
1949      1  0.999657  ablos
...     ...       ...    ...
1229      2  0.000750  zweit
1229      3  0.999140  zweit
1229      4  0.000125  zweit
1598      2  0.999580  zwolf
1598      4  0.000314  zwolf

[610 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 1])

## The names of the players take a huge influence on the prediciton therefore we remove them in the next step

In [ ]:
df_stem['data']

0     trainer alonso mitgereisten fan monaco glucksm...
1     sechser droht viereinhalb monaten paus bitter ...
2     leverkusen sechser erklart budapest spiel jung...
3     bakker mittwoch abschluss train leverkusen nic...
4     warum sehe faz net nicht allerd andrich argent...
                            ...                        
80    bayer leverkusen wehrt europa leagu lang gut s...
81    alonso taktik duell alten lehrmeist jose mouri...
82    trotz pleit halbfin hinspiel rom gibt leverkus...
83    hitzigen aufeinandertreffen stadio olimpico na...
84    hitzigen aufeinandertreffen stadio olimpico na...
Name: data, Length: 60625, dtype: object

In [ ]:
df_stem['data'].iloc[1]

'sechser droht viereinhalb monaten paus bitter abschi leverkusen dafur erscheint verletzungspaus lang zudem konkurrenzsitu doppelsech seit woch voll saft stehenden palacio amiri demirbai andrich voraussetzungen extrem schwierig fur aranguiz zuletzt aufgrund personellen engpass innenverteidigung zentral abwehrkett eingesetzt andrich kunftig mittelfeld auflaufen durft alonso vorrucken jahrigen sech sonntag hertha bsc nicht sicher einstufen mocht'

In [ ]:
lda_2 = models.LdaModel(corpus, num_topics=10, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)

## Finding the best number of topics

In [ ]:
models_arr = {'iteration_' + str(iter): '' for iter in range(2, 15)}
for iter in range(2, 15): 
    models_arr.update({'iteration_'+ str(iter): models.LdaModel(corpus, num_topics=iter, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)})


In [ ]:
models_arr['iteration_5'].show_topics(20)

[(0,
  '0.050*"hincapi" + 0.022*"fur" + 0.021*"alonso" + 0.021*"spiel" + 0.014*"mourinho" + 0.012*"leverkusen" + 0.010*"immer" + 0.009*"minut" + 0.009*"note" + 0.009*"andrich"'),
 (1,
  '0.038*"leagu" + 0.038*"bayer" + 0.038*"europa" + 0.027*"leverkusen" + 0.026*"wirtz" + 0.021*"union" + 0.018*"metern" + 0.017*"bank" + 0.017*"seit" + 0.017*"sekunden"'),
 (2,
  '0.037*"fur" + 0.029*"leverkusen" + 0.026*"nicht" + 0.022*"alonso" + 0.016*"trainer" + 0.015*"bayer" + 0.011*"monaco" + 0.011*"bochum" + 0.010*"rolf" + 0.010*"innenverteidig"'),
 (3,
  '0.021*"fur" + 0.018*"zwei" + 0.018*"sonntag" + 0.013*"gelb" + 0.013*"problem" + 0.013*"nicht" + 0.012*"drei" + 0.011*"leverkusen" + 0.010*"bayer" + 0.010*"weigl"'),
 (4,
  '0.024*"leverkusen" + 0.018*"hincapi" + 0.018*"bayer" + 0.015*"demirbai" + 0.015*"uber" + 0.013*"fur" + 0.011*"tor" + 0.010*"adli" + 0.010*"ball" + 0.009*"diabi"')]

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(models_arr['iteration_5'], corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.118009 -0.075149       1        1  27.391226
2      0.171329 -0.019265       2        1  21.986866
1     -0.241516  0.269830       3        1  19.276767
0     -0.091318 -0.292940       4        1  18.229524
3      0.279514  0.117524       5        1  13.115617, topic_info=           Term          Freq          Total Category  logprob  loglift
546     hincapi  69361.000000   69361.000000  Default  30.0000  30.0000
73       europa  40319.000000   40319.000000  Default  29.0000  29.0000
89        leagu  43421.000000   43421.000000  Default  28.0000  28.0000
193       union  19840.000000   19840.000000  Default  27.0000  27.0000
0        alonso  49365.000000   49365.000000  Default  26.0000  26.0000
..          ...           ...            ...      ...      ...      ...
42        nicht   8172.653173   67191.972780   Topic5  -4.3685  -0.0754
313       diabi   5679.861428   23826.275167   Topic5  -4.7323   0.5975
374      beiden   4187.084655    8364.188106   Topic5  -5.0373   1.3394
38   leverkusen   7135.648823  107629.591162   Topic5  -4.5042  -0.6822
61        bayer   6459.159882   89328.432885   Topic5  -4.6038  -0.5955

[351 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1933      2  0.999655      ,,e
801       2  0.999736    ,,ich
535       2  0.999645    ,,wir
535       3  0.000233    ,,wir
1949      2  0.999671    ablos
...     ...       ...      ...
423       4  0.005588  zweimal
423       5  0.994245  zweimal
1229      1  0.999045    zweit
1229      2  0.000148    zweit
1229      4  0.000739    zweit

[577 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 1, 4])

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(models_arr['iteration_7'], corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.012286 -0.098674       1        1  33.302538
2     -0.190052  0.027614       2        1  14.733970
6      0.374452 -0.196944       3        1  13.234982
3     -0.194652  0.083265       4        1  12.815504
5     -0.060092 -0.150569       5        1  12.544450
4     -0.109657 -0.067780       6        1   7.430198
1      0.192288  0.403088       7        1   5.938358, topic_info=           Term          Freq         Total Category  logprob  loglift
546     hincapi  83020.000000  83020.000000  Default  30.0000  30.0000
61        bayer  73853.000000  73853.000000  Default  29.0000  29.0000
564       wirtz  36976.000000  36976.000000  Default  28.0000  28.0000
0        alonso  53655.000000  53655.000000  Default  27.0000  27.0000
89        leagu  35898.000000  35898.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
373      begann   4363.585176   7584.384100   Topic7  -4.2036   2.2709
544        fosu   3454.950580   7247.199085   Topic7  -4.4371   2.0829
549      mensah   3454.950580   7247.199085   Topic7  -4.4371   2.0829
1    bundesliga   3163.018644   7704.299621   Topic7  -4.5254   1.9335
119         gut   3195.373023   9011.817725   Topic7  -4.5152   1.7869

[326 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
801       6  0.999819       ,,ich
245       2  0.999855        adam
246       1  0.996448        adli
246       2  0.003507        adli
1399      2  0.999696      akteur
...     ...       ...         ...
197       3  0.999857  zuschauern
289       1  0.979176        zwei
289       5  0.020672        zwei
1229      3  0.999964       zweit
1598      7  0.999714       zwolf

[372 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 7, 4, 6, 5, 2])